In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import re

In [2]:
#爬韓星板
def create_ptt_url(page):
    web_url = "https://www.ptt.cc/bbs/KoreaStar/index"
    end = ".html"
    
    return "{0}{1}{2}".format(web_url, page, end)

In [3]:
def get_web_page(url):
    resp = requests.get(url)
    resp.encoding = "utf-8"
    
    if resp.status_code != 200:
        print("Invalid url:", resp.url)
        return None
    else:
        return resp.text

In [4]:
def create_posts_df(dom):
    soup = BeautifulSoup(dom, "html.parser")
    posts = soup.find_all("div", class_ = "r-ent")
    posts_lst = []
    
    for index, row in enumerate(posts):
        push = row.find("div", class_ = "nrec")
        title = push.find_next("div", class_ = "title")
        author = title.find_next("div", class_ = "author")
        date = author.find_next("div", class_ = "date")
    
        title_a = title.text.replace("\n", "").replace("\n", "")
        posts_lst.append([push.text, title_a, author.text, date.text])
        posts_lst1 = np.array(posts_lst)
    
        posts_df = pd.DataFrame(posts_lst1, columns = ["push", "title", "author", "date"])
    
    return posts_df

In [5]:
posts_df_full = pd.DataFrame()
for page in range(1430,1441):
    url = create_ptt_url(page)
    dom = get_web_page(url)
    posts_df = create_posts_df(dom)
    posts_df_full = pd.concat([posts_df_full, posts_df], axis = 0)
    
posts_df_full = posts_df_full.reset_index(drop = True)
posts_df_full

,push,title,author,date
0,83,[介紹] BEG的成人愛豆 - Narsha(流量注意&19禁),jing1219,10/18
1,X9,[新聞] 雪莉出殯穿粉色還比V KAI挨轟「不尊重」粉：道德綁架,nvidia123,10/18
2,22,Re: [新聞] PD101 第一季投票相關（非原標),siulou,10/18
3,爆,[新聞] SJ Time_Slip摘32區iTunes冠軍5天賣35萬張,hvariables,10/18
4,X1,[新聞] IU一直為閨蜜雪莉守靈 因未發文遭網友攻,StressND,10/18
5,60,[新聞] 羅PD後悔了登月球要8千億,StressND,10/18
6,7,[新聞] BOL4首張原創日文單曲 明年1月推出,hvariables,10/18
7,爆,[閒聊] 知道他的年紀後會嚇一跳的演員 李宰旭,isayhousayya,10/18
8,X8,[新聞] 再槓上公司！賴冠霖：到底想要怎樣？,juju1019,10/19
9,20,[新聞] 狗血無極限！林秀香《優雅的家》身世揭,yoyodiy,10/19


In [9]:
#爬留言
dom = get_web_page("https://www.ptt.cc/bbs/KoreaStar/M.1571709380.A.4E4.html")
soup = BeautifulSoup(dom, "html.parser")
push = soup.find_all("div", class_ = "push")
reply_lst = []
for index, row in enumerate(push):
    userid = row.find("span", class_ = "push-userid")
    content = userid.find_next("span", class_ = "push-content")
    content1 = content.text.replace(":", "")
    ipdate = content.find_next("span", class_ = "push-ipdatetime")
    ipdate1 = ipdate.text.replace("\n", "")
    reply_lst.append([userid.text, content1, ipdate1])
    
reply_lst1 = np.array(reply_lst)
reply_df = pd.DataFrame(reply_lst1, columns = ["userid", "content", "ip & date"])


In [10]:
reply_df

,userid,content,ip & date
0,lunasiao,\MAMAMOO/,220.138.50.193 10/22 09:59
1,milk1982,\MAMAMOO/\MAMAMOO/\MAMAMOO/,49.219.185.111 10/22 10:02
2,poohhsiao,\MAMAMOO/,1.168.128.124 10/22 10:03
3,JDFc,天啊！！！,101.137.177.6 10/22 10:05
4,binbin267310,\MAMAMOO/,1.200.214.177 10/22 10:05
5,geemer,推！！！！！！！,223.138.160.217 10/22 10:05
6,KACIRIE,媽木衝阿!!!!!!,218.173.147.165 10/22 10:07
7,ayumi0128,MAMAMOO推啊！,27.247.75.202 10/22 10:07
8,mgwish,\MAMAMOO/,118.161.69.128 10/22 10:08
9,whaleu6,爆推啊啊啊,125.224.38.12 10/22 10:10
